In [11]:
import os
import re
import csv
import collections
import math
from nltk.corpus import stopwords
from nltk.stem.porter import*
from nltk.tokenize import RegexpTokenizer
import nltk
import collections
import string

In [12]:
path = 'C:\\Users\\lm709\\OneDrive\\codes\\IR_lab\\lab3\\20_newsgroups'

In [13]:
def tfidf(tf, df):
    if tf == 0:
        wf = 0
    else:
        wf = 1 + math.log10(tf)
    idf = math.log10((N * 1.0) / (df + 1))
    return wf * idf

In [ ]:
def spimi():
	#Open all the block files
	blockFiles = [open('DISK/' + i) for i in listdir('DISK/') if i.find('DS_Store') == -1]
	#Open the final index file
	indexFile = open('result/index.txt', 'a+') #python3 - (, encoding = "ISO-8859-1")

	firstLines = OrderedDict() # blockId:{term:[postings]}

	for blockFile in blockFiles:
		if 'BLOCK' in blockFile.name:
			blockFileID = getBlockFileID(blockFile)
			term = {}
			line = blockFile.readline()

			if not line == '':
				lineSplit = line.split(':')
				term = lineSplit[0]
				postings = ast.literal_eval(lineSplit[1])
				firstLines[blockFileID] = {term : postings}

	blocksEmpty = False #set to true when nothing is left in blocks

	while not blocksEmpty:
		lowestWordBlockIdTuple = min([[firstLines[i], i] for i in firstLines]) # get lowest term alphabetically, format [{term: [postings list]}, blockId]
		currentTerm = list(lowestWordBlockIdTuple[0].keys())[0]
		blocksWithThisTerm = [blockId for blockId in firstLines if currentTerm in [termKey for termKey in firstLines[blockId]]] # returns the blockIds of the blocks that have the same term at the top of their file
		combinedPostings = sorted(sum([pl[currentTerm] for pl in (firstLines[i] for i in blocksWithThisTerm)], []))
		indexFile.write(str(currentTerm) + ":" + str(combinedPostings) + "\n")

		for blockFileId in blocksWithThisTerm:
			blockFile = getBlockFileById(blockFileId, blockFiles)
			if blockFile: # if blockFile was found
				term = {}
				line = blockFile.readline()
				if not line == '':
					lineSplit = line.split(':')
					term = lineSplit[0]
					postings = ast.literal_eval(lineSplit[1])
					firstLines[blockFileId] = {term : postings}
				#else, remove blockFile from blockFiles list
				else:
					del firstLines[blockFileId]
					blockFiles.remove(getBlockFileById(blockFileId, blockFiles))
			else:
				print("remove")
				blockFiles.remove(getBlockFileById(blockFileId, blockFiles))
		if not blockFiles: #if blockFiles is completely empty
			blocksEmpty = True
	
	return 0

In [ ]:
def getBlockFileID(blockFile):
	fileName = blockFile.name
	fileIDMatched = re.search('BLOCK(\d*).txt', fileName)
	if fileIDMatched:
		return int(fileIDMatched.group(1))

In [ ]:

def getBlockFileById(usedBlockId, blockFiles):
	matchingFiles = [file for file in blockFiles if re.search('BLOCK' + str(usedBlockId), file.name)] # should be only one that matches
	if matchingFiles: # if not empty
		return matchingFiles[0]
	else:
		return False

In [14]:
plist = {}
N = 0
from nltk.tokenize import RegexpTokenizer
cachedStopWords = stopwords.words("english")
for dirpath, dirnames, files in os.walk(path, topdown=True):
    for file in files:
        N += 1
        fr = open(dirpath+'\\'+file, errors='ignore')
        text = fr.read().lower()
        #to_remove = str.maketrans('','',string.punctuation) #string.puncuation is a string which contains english puncuation
        #without_punctuation = text.translate(to_remove)
        word_tokenizer = RegexpTokenizer('[A-Za-z]+')
        terms = word_tokenizer.tokenize(text)
        #terms = [word for word in terms if word not in cachedStopWords]
        tf_td = collections.Counter(terms)
        terms = set(terms)
        for term in terms:
            pair = ((dirpath+'\\'+file), tf_td[term])
            if term in plist:
                plist[term].add(pair)
            else:
                plist[term] = set([pair])
        fr.close()

In [15]:
length = {}
for term, values in plist.items():
    df = len(plist[term])
    for pair in values:
        doc = pair[0]
        tf = pair[1]
        if doc in length:
            length[doc] += tfidf(tf, df) * tfidf(tf, df)
        else:
            length[doc] = tfidf(tf, df) * tfidf(tf, df)
for key in length:
    length[key] = math.sqrt(length[key])

In [16]:
length['C:\\Users\\lm709\\OneDrive\\codes\\IR_lab\\lab3\\20_newsgroups\\20_newsgroups\\alt.atheism\\49960']

70.4715220682423

In [17]:
def cosineScore(q):
    q = q.lower()
    score = {}
    word_tokenizer = RegexpTokenizer('[A-Za-z]+')
    terms_q = word_tokenizer.tokenize(q)
    tf_cnt = collections.Counter(terms_q)
    terms_q = set(terms_q)
    for term in terms_q:
        if term in plist:
            df = len(plist[term])
            for pair in plist[term]:
                tf_td = pair[1]
                tf_tq = tf_cnt[term]
                w_td = tfidf(tf_td, df)                
                w_tq = tfidf(tf_tq, df)
                if pair[0] in score:
                    score[pair[0]] = score[pair[0]] + w_td * w_tq
                else:
                    score[pair[0]] = w_td * w_tq    
    for doc in score:
        score[doc] = (score[doc]*1.0) / (length[doc])     
    score = sorted(score.items(), key=lambda item:item[1], reverse=True)
    k = 0
    for doc in score:
        print(doc[0])
        k = k + 1
        if k == 10:
            break

In [19]:
while True:
    print("*****************************************************************\n")
    print("*****************************************************************\n")
    print("**********************Enter exit to quit*************************")
    query = input('Please enter the content of query (or press enter to quit): ')
    query = query.lower()
    if query == 'exit':
        break
    else:
        cosineScore(query)

*****************************************************************

*****************************************************************

**********************Enter exit to quit*************************
*****************************************************************

*****************************************************************

**********************Enter exit to quit*************************
*****************************************************************

*****************************************************************

**********************Enter exit to quit*************************
*****************************************************************

*****************************************************************

**********************Enter exit to quit*************************
*****************************************************************

*****************************************************************

**********************Enter exit to quit*************************
